In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import argparse
import cv2
import darknet
import numpy as np
from matplotlib import pyplot as plt
from natsort import natsorted, ns

In [3]:

prototxtPath = "C:/Users/Ranjana/Buffalo_assitantship/Leeway/deploy.prototxt"
weightsPath = "C:/Users/Ranjana/Buffalo_assitantship/Leeway/res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector
model = load_model("my_model_facemask.h5")
print("Model loaded")

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-wmltth_x\opencv\modules\dnn\src\caffe\caffe_io.cpp:1121: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "C:/Users/Ranjana/Buffalo_assitantship/Leeway/deploy.prototxt" in function 'cv::dnn::ReadProtoFromTextFile'


In [4]:
def process_images(input_file_path):

    # read the input image dimensions
    image = cv2.imread(input_file_path)
    (h, w) = image.shape[:2]
 
    # construct a blob from the image
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),(104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    net.setInput(blob)
    detections = net.forward()

    # loop over the detections and find as many as possible
    for i in range(0, detections.shape[2]):

        # extract the confidence associated with the detection
        confidence = detections[0, 0, i, 2]

        # filter out detections by setting a threshold value
        if confidence > 0.4:

            # compute the coordinates of the bounding box for the mask
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB ordering, resize it to 150x150, and preprocess it
            face = image[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (150, 150))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            # pass the face through the model to determine if the face has a mask or not
            (mask, withoutMask) = model.predict(face)[0]

            # determine the class label and color to draw the bounding box and text
            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

            # include the probability in the label
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

            # display the label and bounding box rectangle on the output frame
            cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

    # show the output image
    save_path = input_file_path.split(".",1)
    plt.imshow(image)

process_images("image1.jpg")


NameError: name 'net' is not defined